<font size=16>Homework 07: Constraint Programming</font>

# Textbook Problem 12.9-2

Consider the following problem:

Maximize $Z = 5x_1 - x_1^2 + 8x_2 - x_2^2 + 10x_3 - x_3^2 + 15 x_4 - x_4^2 + 20 x_5 - x_5^2$

subject to 

$ x_1 \in \{3,6,12\}, x_2 \in \{3,6\}, x_3 \in \{3,6,9,12\}, x_4 \in \{6,12\}, x_5 \in \{9,12,15,18\}$

$x_1, x_2, x_3, x_4, x_5$ must all be different 

$x_1 + x_3 + x_4 \leq 25$

**(a)** Without doing any optimization, use CP-SAT to make a list of all feasible solutions.  Your code should be easily generalizable (use an abstract approach).  

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [1]:
# solution printer
from ortools.sat.python import cp_model

class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print(f'{v} = {self.Value(v)}', end = ' ')
        print()

    def solution_count(self):
        return self.__solution_count

# nonlinear objective code
from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

# Creates the variables.
num_vars = 5
sets = [[3,6,12], [3,6], [3,6,9,12], [6,12], [9,12,15,18]]
myvars = [f'x{i+1}' for i in range(num_vars)]
mysqvars = [f'xsq{i+1}' for i in range(num_vars)]
x = [model.NewIntVarFromDomain(cp_model.Domain.FromValues(sets[i]), myvars[i]) for i in range(num_vars)]

mins = [min(sets[i]) for i in range(num_vars)]
maxes = [max(sets[i]) for i in range(num_vars)]
xsq = [model.NewIntVar(mins[i]**2, maxes[i]**2,f'xsq{i+1}') for i in range(num_vars)]

# Creates the constraints.
model.Add(x[0] + x[2] + x[3] <= 25)
model.AddAllDifferent(x)
[model.AddMultiplicationEquality(xsq[i], [x[i], x[i]]) for i in range(num_vars)] # NEW to enforce ysq = y*y

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Create a solver and solve.
solution_printer = VarArraySolutionPrinter([x[0], x[1], x[2], x[3], x[4]])
status = solver.SearchForAllSolutions(model, solution_printer)

#Print Results
print(f'Status = {solver.StatusName(status)}')
print(f'Number of solutions found: {solution_printer.solution_count()}')


x1 = 3 x2 = 6 x3 = 9 x4 = 12 x5 = 18 
x1 = 3 x2 = 6 x3 = 9 x4 = 12 x5 = 15 
Status = OPTIMAL
Number of solutions found: 2


**(b)** Now use CP-SAT to solve the optimization problem.  Your code should be easily generalizable (use an abstract approach).  

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [2]:
# solution printer
from ortools.sat.python import cp_model

class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print(f'{v} = {self.Value(v)}', end = ' ')
        print()

    def solution_count(self):
        return self.__solution_count

# nonlinear objective code
from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

# Creates the variables.
num_vars = 5
sets = [[3,6,12], [3,6], [3,6,9,12], [6,12], [9,12,15,18]]
coefs = [5,8,10,15,20] #for obj function
myvars = [f'x{i+1}' for i in range(num_vars)]
mysqvars = [f'xsq{i+1}' for i in range(num_vars)]
x = [model.NewIntVarFromDomain(cp_model.Domain.FromValues(sets[i]), myvars[i]) for i in range(num_vars)]

mins = [min(sets[i]) for i in range(num_vars)]
maxes = [max(sets[i]) for i in range(num_vars)]
xsq = [model.NewIntVar(mins[i]**2, maxes[i]**2,f'xsq{i+1}') for i in range(num_vars)]

# Creates the constraints.
model.Add(x[0] + x[2] + x[3] <= 25)
model.AddAllDifferent(x)
[model.AddMultiplicationEquality(xsq[i], [x[i], x[i]]) for i in range(num_vars)] # NEW to enforce ysq = y*y

# Add an objective function and a direction, need not be linear
#model.Maximize(5*x[0] -xsq[0] + 8*x[1] - xsq[1] +10*x[2] - xsq[2] + 15*x[3] - xsq[3] +20*x[4] - xsq[4])
model.Maximize( sum( coefs[i]*x[i] - 1*xsq[i] for i in range(num_vars)) )

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

#Print results
if status == cp_model.OPTIMAL:
    print('Maximum of objective function: %i' % solver.ObjectiveValue())
    print()
    [print(f'x{i+1} = %i' % solver.Value(x[i])) for i in range(num_vars)]


Maximum of objective function: 138

x1 = 3
x2 = 6
x3 = 9
x4 = 12
x5 = 15


# Assignment Problem

This problem is based on textbook problem 9.1-7 

The Move-It Company has two plants producing forklift trucks that then are shipped to three distribution centers. The production costs are the same at the two plants, and the cost of shipping for each truck is shown for each combination of plant and distribution center: 

<img src="./images/forklift_table.png" width=400>

A total of 60 forklift trucks are produced and shipped per week. Each plant can produce and ship any amount up to a maximum of 50 trucks per week, so there is considerable flexibility on how to divide the total production between the two plants so as to reduce shipping costs. However, each distribution center must receive exactly 20 trucks per week.

The objective of management is to determine how many forklift trucks should be produced at each plant, and then what the overall shipping pattern should be to minimize total shipping cost.

We are going to solve this two different ways:

**(a)** *Allow product splitting.*  Each distribution center can receive forklift trucks from both plants (it's possible distribution center 1 gets 10 from A and 10 from B).    Solve this transportation by adapting abstract Pyomo code from Lesson 3.  Note that the total supply is greater than the total demand so you'll to include a dummy distribution center that receives the excess supply to turn this into a balanced transportation problem.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [3]:
plant = ['PlantA', 'PlantB']
supply = dict(zip(plant, [50, 50]))

distribution = ['DC1', 'DC2', 'DC3', 'Dummy']
demand = dict(zip(distribution, [20, 20, 20, 40]))

littleM = 0
cost_list = [[600, 700, 400, littleM],
             #[bigM, bigM, bigM, bigM],
              [700, 800, 500, littleM]
            #,[bigM, bigM, bigM, bigM]
            ]

cost = {
    plant[p]: {distribution[d]: cost_list[p][d]
                   for d in range(len(distribution))}
    for p in range(len(plant))
}

#throw an error if total supply and demand do not match
assert (sum(supply.values()) == sum(demand.values()))

from pyomo.environ import *

model = ConcreteModel()

model.assign= Var(plant, distribution, domain=NonNegativeReals)

model.total_cost = Objective(expr=sum(cost[p][d] * model.assign[p, d]
                                      for p in plant for d in distribution),
                             sense=minimize)

model.supply_ct = ConstraintList()
for p in plant:
    model.supply_ct.add(
        sum(model.assign[p, d] for d in distribution) == supply[p])

model.demand_ct = ConstraintList()
for d in distribution:
    model.demand_ct.add(
        sum(model.assign[p, d
                        ] for p in plant) == demand[d])

# solve and display
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("Minimum Total Cost per Week = ",
      numbers.format_currency(model.total_cost(), 'USD', locale='en_US'))
# put amounts in dataframe for nicer display
import pandas as pd
dvars = pd.DataFrame([[model.assign[p, d]() for d in distribution]
                      for p in plant],
                     index = plant,
                     columns=distribution)
print("Number of trucks per week to Distribution Centers:")
dvars



Minimum Total Cost per Week =  $35,000.00
Number of trucks per week to Distribution Centers:


,DC1,DC2,DC3,Dummy
PlantA,20.0,20.0,10.0,0.0
PlantB,0.0,0.0,10.0,40.0


**(b)** *No product splitting allowed* . Each distribution center can receive forklift trucks from only one plant (to lower administrative and other hidden costs).   So one plant sends two shipments of 20 to each of two distribution centers, while the other plant sends one shipment of 20 to the remaining distribution center.

Follow the "Formulation of option 2" on page 354 to formulate this as an assignment problem, then use CP-SAT with Element and All-Different constraints to solve this assignment problem to find the minimum cost.  The minimum cost with no-splitting allowed should be larger than the minimum cost when splitting is allowed.  Your code should be easily generalizable.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [7]:
import pandas as pd
from IPython.display import display, HTML
t = 20 #lot size twenty
locations = ['Loc1', 'Loc2', 'Loc3', 'Dummy']
plants = ['Plant1a', 'Plant1b', 'Plant2a', 'Plant2b']
cost_table = [ [600*t, 700*t, 400*t, 0],
         [600*t, 700*t, 400*t, 0],
         [700*t, 800*t, 500*t, 0],
         [700*t, 800*t, 500*t, 0]
       ]


num_locations = len(cost_table[0])
num_plants = len(cost_table)

from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

# Creates the variables.
assign = [
    model.NewIntVar(0, num_locations - 1, plants[i])
    for i in range(num_plants)
]


#get the maximum cost from the cost_table for the top of our intvar range
max_cost = max(list(map(max, cost_table)))
cost = [model.NewIntVar(0, max_cost, f'cost{i}') for i in range(num_plants)]

# Constraints
model.AddAllDifferent(assign)

for i in range(num_locations):
    model.AddElement(assign[i], cost_table[i], cost[i]) #cost[i]= cost_table[i][assign[i]] 

model.Minimize(sum(cost))


# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print(f'Lowest Possible Cost: ${solver.ObjectiveValue()}')
    print()
    print('Assignments and associated costs (note: these are total dollars based on 20 units per lot, at $x per unit, per square on the grid):')
    cost_assigns = pd.DataFrame(0, index=plants, columns=locations)
    for i in range(num_plants):
        cost_assigns.iloc[i, solver.Value(assign[i])] = solver.Value(cost[i])
    display(cost_assigns)

Lowest Possible Cost: $36000.0

Assignments and associated costs (note: these are total dollars based on 20 units per lot, at $x per unit, per square on the grid):


,Loc1,Loc2,Loc3,Dummy
Plant1a,0,14000,0,0
Plant1b,12000,0,0,0
Plant2a,0,0,10000,0
Plant2b,0,0,0,0


# Different Assignment-like Problem

A college student has 7 days remaining before final examinations begin in her four courses, and she wants to allocate this study time as effectively as possible. She needs at least 1 day on each course, and she likes to concentrate on just one course each day, so she wants to allocate 1, 2, 3, or 4 days to each course. Having recently taken an OR course, she decides to use dynamic programming to make these allocations to maximize the total grade points to be obtained from the four courses. She estimates that the alternative allocations for each course would yield the number of grade points shown in the following table:

<img src="./images/grades_table.png" width=400>

Use a CP-SAT constraint programming approach with the Element constraint and other appropriate constraints to maximize the total grade points. Your code should be easily generalizable.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [8]:
import pandas as pd
from IPython.display import display, HTML
from ortools.sat.python import cp_model

courses = ['Course1', 'Course2', 'Course3', 'Course4'] #To test more courses: , 'CourseXX'
dayvalues = ['1 Days', '2 Days', '3 Days', '4 Days']
gpa_table_cols = [ [3,5,6,7], #columns (each set represents a course, from 1 day to 4 days of studying)
               [5,5,6,9],
               [2,4,7,8],
               [6,7,9,9]
               #,[1,1,1,1] To test more courses
             ]

gpa_table = gpa_table_cols #toggle gpa_table_rows/columns

num_courses = len(gpa_table)
num_dayvalues = len(gpa_table[0])


# Create the model.
model = cp_model.CpModel()

###Creates the variables.
assign = [
    model.NewIntVar(0, num_courses - 1, courses[i])
    for i in range(num_courses)
]

max_gpa = max(list(map(max, gpa_table)))
GPA = [model.NewIntVar(0, max_gpa, f'GPA{i}') for i in range(num_courses)]

##Constraints
#model.AddAllDifferent(assign) #assign[0], assign[1], assign[2], assign[3]) #I believe my problem is by its structure picking 'all different'. No need for this?
for i in range(num_courses):
    model.AddElement(assign[i], gpa_table[i], GPA[i]) #cost[i]= cost_table[i][assign[i]]
#limit the number of days to 7 total (4 + 3 vars = 7)
model.Add(assign[0] + 1 + assign[1] + 1 + assign[2] + 1 + assign[3] + 1 == 7) #To test more courses: + assign[4]+1

model.Maximize(sum(GPA))

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)


# Display Results:
if status == cp_model.OPTIMAL:
    print(f'Highest Possible GPA: {solver.ObjectiveValue()}')
    print()
    print('Assignments Yielding Highest GPA:')
    gpa_assigns = pd.DataFrame(0, index=courses, columns= dayvalues)
    for i in range(num_courses):
        gpa_assigns.iloc[i, solver.Value(assign[i])] = 'Study' #solver.Value(GPA[i]) #toggle this to get the actual GPA points for a course/day
    display(gpa_assigns)


Highest Possible GPA: 23.0

Assignments Yielding Highest GPA:


,1 Days,2 Days,3 Days,4 Days
Course1,0,Study,0,0
Course2,Study,0,0,0
Course3,0,0,Study,0
Course4,Study,0,0,0


In [46]:
###Here is a printout of all POSSIBLE solutions. The optimal solution is found within them,
###and displayed above for the original problem Q/A.
# solution printer
from ortools.sat.python import cp_model

class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            #Here I am inserting a "+1" to account for it just being an index...that will give an accurate
            #Value of the total days assigned to each course
            print(f'{v} = {self.Value(v)+1}', end = ' ')
        print()

    def solution_count(self):
        return self.__solution_count
    
# Create a solver and solve.
solution_printer = VarArraySolutionPrinter([assign[0] , assign[1], assign[2], assign[3]]) #To test mre courses: , assign[4]
status = solver.SearchForAllSolutions(model, solution_printer)

#Print Results
print(f'Status = {solver.StatusName(status)}')
print(f'Number of solutions found: {solution_printer.solution_count()}')

Course1 = 1 Course2 = 1 Course3 = 1 Course4 = 4 
Course1 = 1 Course2 = 2 Course3 = 1 Course4 = 3 
Course1 = 1 Course2 = 4 Course3 = 1 Course4 = 1 
Course1 = 1 Course2 = 3 Course3 = 1 Course4 = 2 
Course1 = 1 Course2 = 1 Course3 = 2 Course4 = 3 
Course1 = 1 Course2 = 2 Course3 = 2 Course4 = 2 
Course1 = 1 Course2 = 3 Course3 = 2 Course4 = 1 
Course1 = 1 Course2 = 1 Course3 = 4 Course4 = 1 
Course1 = 1 Course2 = 2 Course3 = 3 Course4 = 1 
Course1 = 1 Course2 = 1 Course3 = 3 Course4 = 2 
Course1 = 4 Course2 = 1 Course3 = 1 Course4 = 1 
Course1 = 3 Course2 = 2 Course3 = 1 Course4 = 1 
Course1 = 3 Course2 = 1 Course3 = 2 Course4 = 1 
Course1 = 3 Course2 = 1 Course3 = 1 Course4 = 2 
Course1 = 2 Course2 = 2 Course3 = 1 Course4 = 2 
Course1 = 2 Course2 = 1 Course3 = 1 Course4 = 3 
Course1 = 2 Course2 = 2 Course3 = 2 Course4 = 1 
Course1 = 2 Course2 = 1 Course3 = 2 Course4 = 2 
Course1 = 2 Course2 = 1 Course3 = 3 Course4 = 1 
Course1 = 2 Course2 = 3 Course3 = 1 Course4 = 1 
Status = OPTIMAL
Num

# Scheduling

Use the generalizable CP-SAT approach to scheduling shown in the lesson to find a schedule of minimum length for the Reliable Construction Company project with activities A-N shown in the table below:

<img src="./images/reliable_table.png" width=600>

This problem is discussed in a supplemental textbook chapter which we've included in the folder with this notebook if you want to know more about it.

Display both the minimized length and the optimal schedule in both text and with a Gantt chart.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [36]:
# generalizable schedule code
import numpy as np

task_duration_dict = {
    'A-Excavate': 2,
    'B-Foundation': 4,
    'C-Rough Wall': 10,
    'D-Roof': 6,
    'E-Exterior Plumbing': 4,
    'F-Interior Plumbing': 5,
    'G-Exterior Siding': 7,
    'H-Exterior Painting': 9,
    'I-Electrical Work': 7,
    'J-Wallboard': 8,
    'K-Flooring': 4,
    'L-Interior Painting': 5,
    'M-Exterior Fixtures': 2,
    'N-Interior Fixtures': 6,
}
task_names = list(task_duration_dict.keys())
num_tasks = len(task_names)
durations = list(task_duration_dict.values())

# for each task we have a list of tasks that must go after
# task:['these','tasks','after']
precedence_dict = {
    'A-Excavate': [],
    'B-Foundation': ['A-Excavate'],
    'C-Rough Wall': ['B-Foundation'],
    'D-Roof': ['C-Rough Wall'],
    'E-Exterior Plumbing': ['C-Rough Wall'],
    'F-Interior Plumbing': ['E-Exterior Plumbing'],
    'G-Exterior Siding': ['D-Roof'],
    'H-Exterior Painting': ['E-Exterior Plumbing','G-Exterior Siding'],
    'I-Electrical Work': ['C-Rough Wall'],
    'J-Wallboard': ['F-Interior Plumbing','I-Electrical Work'],
    'K-Flooring': ['J-Wallboard'],
    'L-Interior Painting': ['J-Wallboard'],
    'M-Exterior Fixtures': ['H-Exterior Painting'],
    'N-Interior Fixtures': ['K-Flooring','L-Interior Painting']
}

task_name_to_number_dict = dict(zip(task_names, np.arange(0, num_tasks)))

horizon = sum(task_duration_dict.values())

from ortools.sat.python import cp_model
model = cp_model.CpModel()

start_vars = [
    model.NewIntVar(0, horizon, name=f'start_{t}') for t in task_names
]
end_vars = [model.NewIntVar(0, horizon, name=f'end_{t}') for t in task_names]

# the `NewIntervalVar` are both variables and constraints, the internally enforce that start + duration = end
intervals = [
    model.NewIntervalVar(start_vars[i],
                         durations[i],
                         end_vars[i],
                         name=f'interval_{task_names[i]}')
    for i in range(num_tasks)
]

# precedence constraints
for before in list(precedence_dict.keys()):
    for after in precedence_dict[before]:
        before_index = task_name_to_number_dict[before]
        after_index = task_name_to_number_dict[after]
        model.Add(end_vars[after_index] <= start_vars[before_index])

obj_var = model.NewIntVar(0, horizon, 'largest_end_time')
model.AddMaxEquality(obj_var, end_vars)
model.Minimize(obj_var)

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f'Optimal Schedule Length: {solver.ObjectiveValue()} weeks')
print(f'Maximum Schedule if all weeks added together: {sum(durations)} weeks')
print()
for i in range(num_tasks):
    print(
        f'{task_names[i]} start at week {solver.Value(start_vars[i])} and end at week {solver.Value(end_vars[i])}'
    )

Optimal Schedule Length: 44.0 weeks
Maximum Schedule if all weeks added together: 79 weeks

A-Excavate start at week 0 and end at week 2
B-Foundation start at week 2 and end at week 6
C-Rough Wall start at week 6 and end at week 16
D-Roof start at week 16 and end at week 22
E-Exterior Plumbing start at week 16 and end at week 20
F-Interior Plumbing start at week 20 and end at week 25
G-Exterior Siding start at week 22 and end at week 29
H-Exterior Painting start at week 29 and end at week 38
I-Electrical Work start at week 16 and end at week 23
J-Wallboard start at week 25 and end at week 33
K-Flooring start at week 33 and end at week 37
L-Interior Painting start at week 33 and end at week 38
M-Exterior Fixtures start at week 38 and end at week 40
N-Interior Fixtures start at week 38 and end at week 44


In [31]:
# use bokeh to make a Gantt chart
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

output_notebook()
starts = [solver.Value(start_vars[i]) for i in range(num_tasks)]
ends = [solver.Value(end_vars[i]) for i in range(num_tasks)]

source = ColumnDataSource(data=dict(tasks=task_names, starts = starts, ends=ends))

p = figure(x_range=(0,solver.ObjectiveValue()), y_range=task_names, plot_height=350, title="Task Time Spans",
           toolbar_location=None, tools="")

p.hbar(y='tasks', left='starts', right='ends', height=0.9, source=source)

p.xaxis.axis_label = "Time"
p.ygrid.grid_line_color = None

show(p)

Loading BokehJS ...